In [1]:
import time
from typing import Callable, Optional, Tuple, Dict, Any, List

from scienceworld import ScienceWorldEnv
import textworld.gym

import numpy as np
from alfworld.agents.environment import get_environment
import alfworld.agents.modules.generic as generic

# Science World

In [2]:
# Pretty-print helpers (optional)
def hr(title: Optional[str] = None):
    print("\n" + "-"*90)
    if title:
        print(title)
        print("-"*90)

def pkey(d: Dict[str, Any], key: str, label: Optional[str] = None):
    if key in d:
        print(f"{label or key}: {d[key]}")

# Environment bootstrap (works with or without a custom JAR)
def init_env(jar_path: Optional[str] = None, env_step_limit: int = 100) -> ScienceWorldEnv:
    """
    If jar_path is None or "", ScienceWorldEnv uses the built-in jar.
    """
    env = ScienceWorldEnv("", jar_path or "", envStepLimit=env_step_limit)
    return env

def list_tasks(env: ScienceWorldEnv) -> List[str]:
    tasks = env.get_task_names()
    for i, t in enumerate(tasks):
        print(f"{i:2d}: {t}")
    return tasks

def load_task(
    env: ScienceWorldEnv,
    task_name: str,
    var_num: int = 0,
    simplifications: Optional[str] = "easy",
    generate_gold_path: bool = True
):
    """
    simplifications: e.g., "easy" or a comma-joined string like "teleportAction,openDoors,selfWateringFlowerPots,noElectricalAction"
    """
    env.load(task_name, var_num, simplifications or "", generateGoldPath=generate_gold_path)
    obs, info = env.reset()
    return obs, info

In [3]:
# Use default envs defined in science world
env = init_env(jar_path=None, env_step_limit=100)
tasks = list_tasks(env)

 0: boil
 1: change-the-state-of-matter-of
 2: chemistry-mix
 3: chemistry-mix-paint-secondary-color
 4: chemistry-mix-paint-tertiary-color
 5: find-animal
 6: find-living-thing
 7: find-non-living-thing
 8: find-plant
 9: freeze
10: grow-fruit
11: grow-plant
12: identify-life-stages-1
13: identify-life-stages-2
14: inclined-plane-determine-angle
15: inclined-plane-friction-named-surfaces
16: inclined-plane-friction-unnamed-surfaces
17: lifespan-longest-lived
18: lifespan-longest-lived-then-shortest-lived
19: lifespan-shortest-lived
20: measure-melting-point-known-substance
21: measure-melting-point-unknown-substance
22: melt
23: mendelian-genetics-known-plant
24: mendelian-genetics-unknown-plant
25: power-component
26: power-component-renewable-vs-nonrenewable-energy
27: test-conductivity
28: test-conductivity-of-unknown-substances
29: use-thermometer


In [4]:
# Choose task to load / play around with
task_idx = 5
task_name = tasks[task_idx]
print(f"Loading task #{task_idx}: {task_name}.")

# Load task and print initial observation
obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=True)
print(f"Task: {env.get_task_description()}")
print(f"Goal Progress: {env.get_goal_progress()}")
gold_actions = env.get_gold_action_sequence()
# print(gold_actions)

# for idx, a in enumerate(gold_actions):
#     print(f"Step {idx+1}:\n{obs}")
#     print(f"Action: {a}")
#     obs, reward, isCompleted, infos = env.step(a)
#     print(f"Reward: {reward:.2f} | Score: {infos['score']:.2f} |  Completed? {isCompleted}\n\n")

Loading task #5: find-animal.
Task: Your task is to find a(n) animal. First, focus on the thing. Then, move it to the green box in the kitchen.
Goal Progress: Completed keys: 
----------------------------------------------------------------------------------------------------
Sequential Subgoals:
----------------------------------------------------------------------------------------------------
0	false	                       GoalFocusOnAnimal	focus on an animal
1	false	                   GoalObjectInContainer	move living thing to answer box
----------------------------------------------------------------------------------------------------
Unordered and Optional Subgoals:
----------------------------------------------------------------------------------------------------
0	false	                   GoalObjectInContainer	Pick up object
1	true	                  GoalInRoomWithOpenDoor	Be in a room with an open door
2	false	                   GoalMoveToNewLocation	Move to a new location
3	

In [ ]:
play_scienceworld = True

if play_scienceworld:
    # Load task    
    task_idx = 12
    task_name = tasks[task_idx]
    print(f"Loading task #{task_idx}: {task_name}.")
    obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=False)
    print(f"Task: {env.get_task_description()}")
    score, moves, done = 0, 0, False
    
    # Render loop
    while not done:
        command = input("> ")
        obs, score, done, infos = env.step(command)
        print(obs)
        print(score)
        moves += 1

    env.close()
    print("moves: {}; score: {}".format(moves, score))

Loading task #12: identify-life-stages-1.
Task: Your task is to focus on the 3 life stages of the wolf, starting from earliest to latest.
This room is called the art studio. In it, you see: 
	the agent
	a substance called air
	a large cupboard. The large cupboard door is closed. 
	a table. On the table is: a bowl (containing nothing).
	a wood cup (containing yellow paint)
	a wood cup (containing blue paint)
	a wood cup (containing red paint)
You also see:
	A door to the hallway (that is open)
No known action matches that input.
Possible actions: 
	activate OBJ
	close OBJ
	deactivate OBJ
	dunk OBJ in OBJ
	eat OBJ
	flush OBJ
	focus on OBJ
	go OBJ
	inventory
	look around
	look at OBJ
	look in OBJ
	mix OBJ
	move OBJ to OBJ
	open OBJ
	pick up OBJ
	pour OBJ in OBJ
	put down OBJ
	read OBJ
	reset task
	task
	teleport OBJ
	use OBJ on OBJ
	wait
	wait1
You move to the hallway.
This room is called the hallway. In it, you see: 
	the agent
	a substance called air
	a painting
You also see:
	A door to

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42681)
Traceback (most recent call last):
  File "/home/lucas/code/seq-decision/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22873/2356589182.py", line 14, in <module>
    command = input("> ")
              ^^^^^^^^^^^
  File "/home/lucas/code/seq-decision/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 1275, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/lucas/code/seq-decision/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 1320, in _input_request
    raise KeyboardInterrupt(msg) from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lucas/code/seq-decision/.venv/lib/python3.1

KeyboardInterrupt: Interrupted by user

# Text World

In [ ]:
!rm -f tw_games/*
!tw-make custom --world-size 5 --nb-objects 10 --quest-length 5 --output tw_games/custom_game.z8


env_id = textworld.gym.register_game("tw_games/custom_game.z8",
                                     max_episode_steps=50)
env = textworld.gym.make(env_id)
obs, infos = env.reset()
env.render()

Global seed: 44325
Game generated: /home/lucas/code/seq-decision/sequential-decision-processors/data/textworld/raw/tw_games/custom_game.z8




                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
    

In [ ]:
play_textworld = False

if play_textworld:
    score, moves, done = 0, 0, False
    while not done:
        command = input("> ")
        obs, score, done, infos = env.step(command)
        env.render()
        moves += 1

    env.close()
    print("moves: {}; score: {}".format(moves, score))

> look

-= Attic =-
You've entered an attic. You can barely contain your excitement.

You can make out a locked display.

There is a closed passageway leading west. There is an open portal leading east.


>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> 

I beg your pardon?

>
-= Attic =-0/2

> look

-= Attic =-
You've entered an attic. You can barely contain your excitement.

You can make out a locked display.

There is a closed passageway leading west. There is an open portal leading east.


>
-= Attic =-0/3

> go east


-= Workshop =-
You've entered a workshop.

Look over there! an armchair. The armchair is ordinary. The armchair appears to
be empty.

There is an open portal leading west. You don't like doors? Why not try going
north, that entranceway is unguarded